In [1]:
import re
import random

import fasttext
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
tqdm.pandas()

/home/abdullah/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
sentences = pd.read_csv("data_pp/sentences.csv")
sentences_ann_new = pd.read_csv("data_pp/sentences_annotated_new.csv")
collections = pd.read_csv("data_pp/collections.csv")


In [11]:
collections["lead_id"].isin(sentences["lead_id"]).sum()

4924

In [4]:
len(sentences), len(sentences_ann_new)

(355715, 355715)

In [5]:
# get the model
# https://fasttext.cc/docs/en/language-identification.html
# https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
path_to_pretrained_model = 'lid.176.bin'
fmodel = fasttext.load_model(path_to_pretrained_model)
def lang_detect_ft(doc):
    if isinstance(doc, str):
        doc = re.sub("\s+", " ", doc)
        return fmodel.predict([doc])[0][0][0][len("__label__"):]
    return np.nan

In [6]:
sentences["lang_ft"] = sentences["excerpt_text"].progress_apply(lang_detect_ft)

In [7]:
sentences["lang"] = sentences["lang_ft"].copy()
for row in tqdm(sentences.iterrows(), total=len(sentences)):
    idx = row[0]
    if row[1].lang_ft not in ["es", "en", "fr"]:
        if row[1].project_country_code == "COL":
            sentences.loc[idx, "lang"] = "es"
        elif row[1].project_country_code in ["SYRDL,SYR", "SYR", "BGD", "NGA"]:
            sentences.loc[idx, "lang"] = "en"
        else:
            sentences.loc[idx, "lang"] = "fr"

In [8]:
sentences["lang"].value_counts()

en    236961
es     59590
fr     59164
Name: lang, dtype: int64

In [9]:
sentences.to_csv("data_pp/sentences_lang.csv", index=None)